# Python Data Model

## Args and Kwargs

In [270]:
def sumlist(a=1, *nums):
    return nums[0] + sumlist(*nums[1:]) if len(nums) > 1 else nums[0]

In [301]:
args = [1,2,4,5]
sumlist(*args) 

7

In [302]:
def shopping_total(**shopping):
    shopping['total'] = sumlist(*shopping.values())
    return shopping

In [303]:
shopping_total(eggs=5, bread = 10)

{'eggs': 5, 'bread': 10, 'total': 10}

In [269]:
sumlist

<function __main__.sumlist(*nums)>

In [271]:
sumlist.__defaults__, sumlist.__code__, sumlist.__code__.co_varnames

((1,),
 <code object sumlist at 0x2b71aea80390, file "<ipython-input-270-04d7365cbcc2>", line 1>,
 ('a', 'nums'))

In [272]:
from inspect import getsource
print(getsource(sumlist))

def sumlist(a=1, *nums):
    return nums[0] + sumlist(*nums[1:]) if len(nums) > 1 else nums[0]



In [273]:
from dis import dis
dis(sumlist)

  2           0 LOAD_GLOBAL              0 (len)
              2 LOAD_FAST                1 (nums)
              4 CALL_FUNCTION            1
              6 LOAD_CONST               1 (1)
              8 COMPARE_OP               4 (>)
             10 POP_JUMP_IF_FALSE       36
             12 LOAD_FAST                1 (nums)
             14 LOAD_CONST               2 (0)
             16 BINARY_SUBSCR
             18 LOAD_GLOBAL              1 (sumlist)
             20 LOAD_FAST                1 (nums)
             22 LOAD_CONST               1 (1)
             24 LOAD_CONST               0 (None)
             26 BUILD_SLICE              2
             28 BINARY_SUBSCR
             30 CALL_FUNCTION_EX         0
             32 BINARY_ADD
             34 RETURN_VALUE
        >>   36 LOAD_FAST                1 (nums)
             38 LOAD_CONST               2 (0)
             40 BINARY_SUBSCR
             42 RETURN_VALUE


## Class

In [305]:
a = [1,2,4,5]
len(a), a.__len__()   

(4, 4)

In [306]:
from itertools import zip_longest

class polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
        self.order = len(self.coeffs) - 1
        
    def __len__(self):
        return self.order + 1
    
    def __repr__(self):
        return f'{self.coeffs[0]}x^{self.order} + ' + polynomial(*self.coeffs[1:]).__repr__() if self.order > 0 else f'{self.coeffs[0]}'
    
    def __add__(self, other):
        new_coeffs = (x + y for x, y in zip_longest(reversed(self.coeffs), reversed(other.coeffs), fillvalue=0))
        return polynomial(*reversed(list(new_coeffs)))
    
    def __call__(self, at):
        return self.coeffs[0]*at**(self.order) + polynomial(*self.coeffs[1:])(at) if self.order > 0 else self.coeffs[0]

In [307]:
p1 = polynomial(1,2,3)
p2 = polynomial(4,5)

In [308]:
p1.coeffs, p1.order

((1, 2, 3), 2)

In [309]:
len(p2)

2

In [310]:
p1

1x^2 + 2x^1 + 3

In [311]:
p1 + p2

1x^2 + 6x^1 + 8

In [312]:
p1(2.1)

11.61

## Decorators

In [317]:
def add(x,y=10):
    return x + y

In [318]:
from time import time

In [319]:
before = time()
add(1,2)
after = time()
print(f'elapsed: {after - before}')

elapsed: 0.00020122528076171875


In [320]:
def timer(func, *args, **kwargs):
    before = time()
    rv = func(*args, **kwargs)
    after = time()
    print(f'elapsed: {after - before}')
    return rv

timer(add,1,2)

elapsed: 1.430511474609375e-06


3

In [321]:
add(6,7)

13

In [322]:
def timer(func, *args):
    def inner(*args):
        before = time()
        rv = func(*args)
        after = time()
        print(f'elapsed {func.__name__}: {after - before}')
        return rv
    return inner

add = timer(add)

add(1,2)

elapsed add: 1.6689300537109375e-06


3

In [323]:
@timer
def add(x,y=10):
    return x + y
@timer
def sub(x,y):
    return x-y

add(6,7), sub(10,20)

elapsed add: 1.6689300537109375e-06
elapsed sub: 2.384185791015625e-06


(13, -10)

## Generators

In [291]:
from scipy.integrate import quad

In [292]:
quad(polynomial(1,1,1,1,1), -2, 2)

(22.133333333333333, 2.4572936278370133e-13)

In [293]:
def rep(x, n):
    return [x for _ in range(n)]

In [294]:
rep(1, 10)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [295]:
def compute(N):
    rv = []
    for n in range(N):
        coeffs = rep(1,n+1)
        rv.append(quad(polynomial(*coeffs), -2, 2))
    return rv

compute(10)

[(4.0, 4.440892098500626e-14),
 (4.0, 5.557335361613839e-14),
 (9.333333333333332, 1.0362081563168127e-13),
 (9.333333333333332, 1.463037600738554e-13),
 (22.133333333333333, 2.4572936278370133e-13),
 (22.133333333333333, 3.839506718631404e-13),
 (58.70476190476191, 6.517537832180443e-13),
 (58.70476190476191, 1.0949432284556056e-12),
 (172.48253968253968, 1.914940869013778e-12),
 (172.4825396825397, 3.36897185512911e-12)]

In [296]:
class computeclass:
    def __call__(self, N):
        rv = []
        for n in range(N):
            coeffs = rep(1,n+1)
            rv.append(quad(polynomial(*coeffs), -2, 2))
        return rv
    
compute = computeclass()
compute(10)

[(4.0, 4.440892098500626e-14),
 (4.0, 5.557335361613839e-14),
 (9.333333333333332, 1.0362081563168127e-13),
 (9.333333333333332, 1.463037600738554e-13),
 (22.133333333333333, 2.4572936278370133e-13),
 (22.133333333333333, 3.839506718631404e-13),
 (58.70476190476191, 6.517537832180443e-13),
 (58.70476190476191, 1.0949432284556056e-12),
 (172.48253968253968, 1.914940869013778e-12),
 (172.4825396825397, 3.36897185512911e-12)]

In [297]:
class computeclass:
    def __init__(self, N):
        self.n = 0
        self.N = N
    
    def __iter__(self):
        return self
        
    def __next__(self):
        self.n += 1
        if self.n > self.N:
            raise StopIteration()
        coeffs = rep(1,self.n)
        return quad(polynomial(*coeffs), -2, 2) 
    
compute = computeclass(N)
[x for x in compute]

[(4.0, 4.440892098500626e-14),
 (4.0, 5.557335361613839e-14),
 (9.333333333333332, 1.0362081563168127e-13),
 (9.333333333333332, 1.463037600738554e-13),
 (22.133333333333333, 2.4572936278370133e-13),
 (22.133333333333333, 3.839506718631404e-13),
 (58.70476190476191, 6.517537832180443e-13),
 (58.70476190476191, 1.0949432284556056e-12),
 (172.48253968253968, 1.914940869013778e-12),
 (172.4825396825397, 3.36897185512911e-12)]

In [298]:
compute = computeclass(N)

In [299]:
next(compute), next(compute), next(compute), next(compute), next(compute), next(compute), next(compute) 

((4.0, 4.440892098500626e-14),
 (4.0, 5.557335361613839e-14),
 (9.333333333333332, 1.0362081563168127e-13),
 (9.333333333333332, 1.463037600738554e-13),
 (22.133333333333333, 2.4572936278370133e-13),
 (22.133333333333333, 3.839506718631404e-13),
 (58.70476190476191, 6.517537832180443e-13))

In [ ]:
# for x in range(10):
#     pass

# xi = iter(range(10))        # xi -> __iter__()
# while True:
#     x = next(xi)            # x  -> __next__()

In [300]:
def compute(N):
    for n in range(N):
        coeffs = rep(1,n+1)
        yield quad(polynomial(*coeffs), -2, 2)
    

[x for x in compute(10)]

[(4.0, 4.440892098500626e-14),
 (4.0, 5.557335361613839e-14),
 (9.333333333333332, 1.0362081563168127e-13),
 (9.333333333333332, 1.463037600738554e-13),
 (22.133333333333333, 2.4572936278370133e-13),
 (22.133333333333333, 3.839506718631404e-13),
 (58.70476190476191, 6.517537832180443e-13),
 (58.70476190476191, 1.0949432284556056e-12),
 (172.48253968253968, 1.914940869013778e-12),
 (172.4825396825397, 3.36897185512911e-12)]

### Context managers are simply generators

In [ ]:
# import pickle
# with open('./data.pickle', 'wb') as picklein:
#     pickle.dump(p1, picklein)

### [James Powell's PyData Seattle 2017 Talk](https://www.youtube.com/watch?v=cKPlPJyQrt4)